In [ ]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft
from scipy.io.wavfile import write
import pyaudio
from matplotlib import cm
from matplotlib.patches import Circle
import matplotlib.cm as cm

cmap = cm.get_cmap("rainbow")


%matplotlib qt


fig, ax = plt.subplots()

p = pyaudio.PyAudio()

chunk = 20000
format = pyaudio.paInt16  
channels = 1  
rate = 44000
duration = .001
numChunks = int(rate / chunk * duration)
chunks = np.array(range(0, numChunks))
stream = p.open(format=format, channels=channels, rate=rate, input=True, frames_per_buffer=chunk)
numSamples = len(chunks)*chunk

samples = np.linspace(0, duration, num = numSamples)

import librosa

def decompose(F):
    m  = librosa.hz_to_midi(F)
    m0 = np.floor(m).astype(int)
    m1 = m0 + 1
    f0 = librosa.midi_to_hz(m0)
    f1 = librosa.midi_to_hz(m1)
    β  = (F - f0) / (f1 - f0)
    return {
        'alpha': 1 - β,
        'beta':  β,
        'note1': librosa.midi_to_note(m0),
        'note2': librosa.midi_to_note(m1)
    }


def noteToColor(note):
    denom = 11
    colorNumber = 0
    if note == "C":
        colorNumber = 0/denom

    elif note == "C#":
        colorNumber = 1/denom

    elif note == "D":
        colorNumber = 2/denom

    elif note == "D#":
        colorNumber = 3/denom
    
    elif note == "E":
        colorNumber = 4/denom

    elif note == "F":
        colorNumber = 5/denom

    elif note == "F#":
        colorNumber = 6/denom

    elif note == "G":
        colorNumber = 7/denom

    elif note == "G#":
        colorNumber = 8/denom

    elif note == "A":
        colorNumber = 9/denom

    elif note == "A#":
        colorNumber = 10/denom

    elif note == "B":
        colorNumber = 11/denom

    color = cmap(colorNumber)

    return color
    
    


def freqToColor(frequency):
    '''So this will take frequency to note then note to color
    we find distance between two notes, note 1 and note 2. The freq is in between two notes 
    alpha * note 1 freq + beta * note 2 freq = the frequency'''
    return decompose(frequency)

def blend_rgba(rgba1, rgba2, a1, a2):
    r1, g1, b1 = rgba1
    r2, g2, b2 = rgba2

    out_a = a2 + a1 * (1 - a2)
    if out_a == 0:
        return (0, 0, 0, 0)

    out_r = (r2 * a2 + r1 * a1 * (1 - a2)) / out_a
    out_g = (g2 * a2 + g1 * a1 * (1 - a2)) / out_a
    out_b = (b2 * a2 + b1 * a1 * (1 - a2)) / out_a

    return (out_r, out_g, out_b, out_a)

i = 0


cir = Circle((0,0), fill = True)
ax.add_patch(cir)
def update(frame):
    global i, intData, colorList, maxR, cir
    
    chunkHere = np.frombuffer(stream.read(chunk), dtype=np.int16)

    
    fourier = fft(chunkHere)
    mag = abs(fourier[:int(len(fourier)/2)])
    indexOfMax = np.argmax(mag)
    frequencies = np.fft.fftfreq(chunk, d=1/rate)[:int(len(fourier)/2)]
    domFreq = frequencies[indexOfMax]
    if domFreq < 20:
        domFreq = 20
    elif domFreq > 20000:
        domFreq = 20000

    out = freqToColor(domFreq)
    note1 = out["note1"][:-1]
    note2 = out["note2"][:-1]
    #print(note1)
    #print(note2)
    color1 = noteToColor(note1)
    color2 = noteToColor(note2)
    alpha = out["alpha"]
    beta = out["beta"]
    print(note1, "1", alpha)
    print(note2, "2", beta)
   # print(alpha)
   # print(beta)
    #colr = tuple((alpha*np.array(color1) + beta*np.array(color2)))
    #colr = blend_rgba(color1[:3], color2[:3], alpha, beta)
   # colr = alpha*color1 + beta*color2
    if alpha > beta:
        colr = color1
    else:
        colr = color2
    cir.set_color(colr)
    #cir.set_radius(mag[indexOfMax])
   


anim = animation.FuncAnimation(fig = fig, func = update, interval = 1)

plt.show()


C:\Users\jshen\AppData\Local\Temp\ipykernel_2748\41174380.py:11: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap("rainbow")
C:\Users\jshen\AppData\Local\Temp\ipykernel_2748\41174380.py:162: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  anim = animation.FuncAnimation(fig = fig, func = update, interval = 1)


B 1 0.4375921165366339
C 2 0.5624078834633661
F 1 0.28384707780766916
F♯ 2 0.7161529221923308
G♯ 1 0.6236003810787006
A 2 0.37639961892129936
A 1 0.32731385019576875
A♯ 2 0.6726861498042312
B 1 0.4375921165366339
C 2 0.5624078834633661
F 1 0.28384707780766916
F♯ 2 0.7161529221923308
G♯ 1 0.17817153745105663
A 2 0.8218284625489434
C 1 0.4231280572988402
C♯ 2 0.5768719427011598
A♯ 1 0.9914775467362482
B 2 0.008522453263751777
F 1 0.25736172030419713
F♯ 2 0.7426382796958029
C♯ 1 0.9989461910679038
D 2 0.0010538089320962147
B 1 0.4375921165366339
C 2 0.5624078834633661
A♯ 1 0.6740119580877664
B 2 0.32598804191223363
F 1 0.6546420828562689
F♯ 2 0.34535791714373115
C♯ 1 0.19807916468515085
D 2 0.8019208353148491
G♯ 1 0.17817153745105663
A 2 0.8218284625489434
A♯ 1 0.3565463694392845
B 2 0.6434536305607155
D♯ 1 0.9311904412479005
E 2 0.06880955875209947
A♯ 1 0.557442562255902
B 2 0.44255743774409795
C♯ 1 0.9989461910679038
D 2 0.0010538089320962147
F 1 0.9194956578909833
F♯ 2 0.08050434210901